In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

print("TensorFlow:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices("GPU"))

# =========================================================
# PATHS
# =========================================================
DATA_ROOT   = "/content/drive/MyDrive/gradcam/extracted_60_samples"
VGG_PATH    = "/content/drive/MyDrive/gradcam/models/allVGG16.h5"
RESNET_PATH = "/content/drive/MyDrive/gradcam/models/allResNet50.h5"
OUT_ROOT    = "/content/drive/MyDrive/gradcam/results_gradcam_only"

IMG_H, IMG_W = 256, 256
N_CHANNELS   = 10

LABELS = ["n", "o", "r"]
LABEL_NAME = {"n": "normal", "o": "overload", "r": "recovery"}
NAME_TO_CHAR = {"normal": "n", "overload": "o", "recovery": "r"}

for cls in ["normal", "overload", "recovery"]:
    os.makedirs(os.path.join(OUT_ROOT, cls), exist_ok=True)

encoder = LabelEncoder()
encoder.fit(LABELS)

# =========================================================
# Load fixed files
# =========================================================
def list_class_files():
    out = {}
    for cls in ["normal", "overload", "recovery"]:
        files = sorted(glob.glob(os.path.join(DATA_ROOT, cls, "*.npy")))
        print(f"{cls}: {len(files)} files")
        out[cls] = files
    return out

files_by_class = list_class_files()

# =========================================================
# Conv layer selector
# =========================================================
def pick_best_conv_layer(model):
    layers = model.layers
    conv_idxs = [i for i,l in enumerate(layers) if isinstance(l, tf.keras.layers.Conv2D)]
    pool_idxs = [i for i,l in enumerate(layers) if isinstance(l, tf.keras.layers.MaxPooling2D)]

    if len(pool_idxs) > 0:
        last_pool = pool_idxs[-1]
        conv_before = [i for i in conv_idxs if i < last_pool]
        if len(conv_before) > 0:
            return layers[conv_before[-1]].name

    return layers[conv_idxs[-1]].name

def build_grad_model(model, conv_layer_name):
    conv_layer = model.get_layer(conv_layer_name)
    return tf.keras.Model(inputs=model.inputs,
                          outputs=[conv_layer.output, model.output])

# =========================================================
# Grad-CAM
# =========================================================
def gradcam_heatmap(grad_model, x_1hwc, class_index):
    x = tf.convert_to_tensor(x_1hwc, dtype=tf.float32)

    with tf.GradientTape() as tape:
        conv_out, preds = grad_model(x, training=False)
        score = preds[:, class_index]

    grads = tape.gradient(score, conv_out)
    weights = tf.reduce_mean(grads, axis=(1, 2))

    conv_out = conv_out[0]
    weights  = weights[0]

    heatmap = tf.reduce_sum(conv_out * weights, axis=-1)
    heatmap = tf.nn.relu(heatmap)

    heatmap = tf.image.resize(
        heatmap[..., tf.newaxis], (IMG_H, IMG_W)
    )
    return tf.squeeze(heatmap, axis=-1).numpy()

# =========================================================
# Plot: 10 strain channels + Grad-CAM
# =========================================================
def save_gradcam_figure(X, h_cam, vmin, vmax, suptitle, save_path):
    fig = plt.figure(figsize=(24, 6), dpi=200)
    gs = fig.add_gridspec(2, N_CHANNELS, height_ratios=[1.0, 0.7],
                          wspace=0.05, hspace=0.25)

    # ---- Top row: strain channels
    axes_top = []
    im_ref = None
    for c in range(N_CHANNELS):
        ax = fig.add_subplot(gs[0, c])
        axes_top.append(ax)
        im = ax.imshow(X[:, :, c], cmap="jet", vmin=vmin, vmax=vmax)
        if im_ref is None:
            im_ref = im
        ax.set_title(f"Ch {c+1}", fontsize=10)
        ax.axis("off")

    fig.colorbar(im_ref, ax=axes_top, fraction=0.015, pad=0.01)

    # ---- Bottom row: Grad-CAM
    ax = fig.add_subplot(gs[1, :])
    im = ax.imshow(h_cam, cmap="jet")
    ax.set_title("Grad-CAM", fontsize=12)
    ax.axis("off")
    fig.colorbar(im, ax=ax, fraction=0.025, pad=0.01)

    fig.suptitle(suptitle, fontsize=14, y=0.98)
    fig.savefig(save_path, bbox_inches="tight")
    plt.close(fig)

# =========================================================
# Run one model
# =========================================================
def run_model(model_path, model_tag):
    print("\n==============================")
    print("Running:", model_tag)
    print("==============================")

    tf.keras.backend.clear_session()
    model = load_model(model_path)
    model.trainable = False

    conv_name = pick_best_conv_layer(model)
    print("Using conv layer:", conv_name)
    grad_model = build_grad_model(model, conv_name)

    total = 0
    for true_folder in ["normal", "overload", "recovery"]:
        files = files_by_class[true_folder]

        for i, fp in enumerate(files):
            base = os.path.splitext(os.path.basename(fp))[0]
            print(f"[{model_tag}][{true_folder}] {i+1}/{len(files)}")

            X = np.load(fp).astype(np.float32)
            x_in = X[np.newaxis, ...]

            probs = model.predict(x_in, verbose=0)[0]
            pred_idx  = int(np.argmax(probs))
            pred_char = encoder.inverse_transform([pred_idx])[0]
            pred_name = LABEL_NAME[pred_char]

            h_cam = gradcam_heatmap(grad_model, x_in, pred_idx)

            vmin, vmax = float(X.min()), float(X.max())
            suptitle = (
                f"{model_tag} | True: {true_folder} | "
                f"Pred: {pred_name} | P={probs[pred_idx]:.3f}"
            )

            save_path = os.path.join(
                OUT_ROOT, true_folder,
                f"{model_tag}__{base}__true-{true_folder}__pred-{pred_name}__gradcam.png"
            )

            save_gradcam_figure(X, h_cam, vmin, vmax, suptitle, save_path)
            total += 1

    print(f"✅ {model_tag}: saved {total} figures")

# =========================================================
# Run models
# =========================================================
run_model(VGG_PATH, "VGG")
run_model(RESNET_PATH, "ResNet50")

print("\n✅ All Grad-CAM results saved to:", OUT_ROOT)
